In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
np.random.seed(42)
plt.style.use('fivethirtyeight')
sns.set()
sns.set_context("talk")
%matplotlib inline

# Univariate Linear Regression
First, we will model linear regression based on a data set that contains housing data (USA_Housing.csv). 

In [ ]:
import pandas as pd
df = pd.read_csv('/srv/shared/USA_Housing.csv')
if 'Address' in df.columns:
    df_adjusted = df.drop(df.columns[-1], axis=1) #remove address col, fix col headers
else:
    df_adjusted = df
df_adjusted

In [ ]:
from sklearn.model_selection import train_test_split
df_training, df_testing = train_test_split(df_adjusted, test_size=0.1, random_state=42)
df_training = df_adjusted.drop(df_testing.index)
df_testing

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled_values = []
X_scaled_values.append(scaler.fit_transform(df_training.iloc[:,[0]]))
X_scaled_values.append(scaler.fit_transform(df_training.iloc[:,[1]]))
X_scaled_values.append(scaler.fit_transform(df_training.iloc[:,[2]]))
X_scaled_values.append(scaler.fit_transform(df_training.iloc[:,[3]]))
X_scaled_values.append(scaler.fit_transform(df_training.iloc[:,[4]]))
Y_scaled_values = scaler.fit_transform(df_training.iloc[:,[5]])
x = X_scaled_values
y = Y_scaled_values

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.scatter(x[1], y, alpha = 0.5, s = 20)
m, b = np.polyfit(x[1].flatten(), y, 1) #np.polyfit assigns to two vars
plt.plot(x[1], m*x[1]+b, color ='red') #mx[i]+b fits linear model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.scatter(x[2], y, alpha = 0.5, s = 20)
m, b = np.polyfit(x[2].flatten(), y, 1)
plt.plot(x[2], m*x[2]+b, color ='red')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.scatter(x[0], y, alpha = 0.5, s = 20)
m, b = np.polyfit(x[0].flatten(), y, 1)
plt.plot(x[0], m*x[0]+b, color ='red')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.scatter(x[3], y, alpha = 0.5, s = 20)
m, b = np.polyfit(x[3].flatten(), y, 1)
plt.plot(x[3], m*x[3]+b, color ='red')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.scatter(x[4], y, alpha = 0.5, s = 20)
m, b = np.polyfit(x[4].flatten(), y, 1)
plt.plot(x[4], m*x[4]+b, color ='red')

> Based on my plots, income and the house age are good features for a linear regressional model as they seem to best fit a linear trend and have less variance than the other features. 

# Manual Exploration of Linear Regression Line

The goal now is to fit a line 
$$
h(\theta) = \theta_0 + \theta_1*x 
$$
to all data points (x,y), such that the L2 error 
$$
E(\theta) = \sum(h(\theta)-y)^2 $$ is minimized. We need to manually change the values of theta0 and theta1 such that we obtain the smallest possible error. 


In [ ]:
def h(theta0, theta1, x):
    """
    Return the model theta0 + theta1*x
    """
    return theta0 + theta1*x

In [ ]:
import numpy as np
def sqerror(x, y, theta0, theta1):
    """
    Input: parameters theta0 and theta1 of the model
    Input: x, y vectors
    Returns: average L2 square error
    Assumptions: none
    """
    res = ((h(theta0, theta1, x) - y)**2).mean()
    return res
print(sqerror(x[0], y, 0.29,0.52))
print(sqerror(x[1], y, 0.29,0.52))

In [ ]:
import numpy as np
import math
def abserror(x, y, theta0, theta1):
    """
    Input: parameters theta0 and theta1 of the model 
    Input: x, y vectors
    Returns: average L1 error
    Assumptions: none
    """
    res = (abs(y - h(theta0, theta1, x))).mean()
    return res
print(abserror(x[0], y, 0.29,0.52))
print(abserror(x[1], y, 0.29,0.52))

In [ ]:
import numpy as np
import math
def huberror(x, y, theta0, theta1, delta):
    """
    Input: parameters theta0, theta1 and delta of the model 
    Input: x, y vectors
    Returns: psuedo huber error
    Assumptions: none
    """
    a = abs(h(theta0, theta1, x) - y)
    return np.mean(delta**2 * (np.sqrt(1 + (a/delta)**2) - 1))
print(huberror(x[0], y, 0.29,0.52,0.1))
print(huberror(x[1], y, 0.29,0.52,0.1))

In [ ]:
from ipywidgets import interact
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
import pylab
import numpy
def f(theta0, theta1):
    """
    Plot the line and points in an interactive panel
    """
    y1 = h(theta0, theta1, x[0]) 
    pylab.plot(x[0], y1) 
    sqerr = round(sqerror(x[0], y, theta0, theta1), 5)
    abserr = round(abserror(x[0], y, theta0, theta1), 5)
    huber = round(huberror(x[0], y, theta0, theta1, 0.01),4)
    pylab.title('L1=' + str(abserr) + '  L2=' + str(sqerr) + '  hub=' + str(huber))
    x1 = x[0]   
    y1 = Y_scaled_values
    pylab.scatter(x1, y1, alpha = 0.5)
    pylab.show() # show the plot  
interact(f, theta1=(0,1,0.1), theta0=(0,1,0.1));

In [ ]:
theta0 = 0.5
theta1 = 0.2
error = 0.04266
theta0 = 0.2
theta1 = 0.4
error =  0.1067
theta0 = 0.29
theta1 = 0.47
error = 0.0089

# Gradient Descent - Univariate
In this task we use the Gradient descent methods to find a "better" values for theta0 and theta1 that minimizes the error. Gradient descent is an iterative algorithm. It computes values of theta0 and theta1 in the direction of reaching the minimum point in the error function. The iterative formulas using L2 loss function for theta0 and theta1 are given by:
$$
\theta_0 = \theta_0 - \alpha*(\sum(\theta_1*x^j + \theta_0)-y^j)
$$
$$
\theta_1 = \theta_1 - \alpha*(\sum(\theta_1*x^j + \theta_0 - y^j)*x^j
$$

The alpha is called the "learning rate". $(x^j, y^j)$ is the j-th observation. It is important to pick a good value for alpha so that convergence is not too slow (small alpha) or be at the risk of over shooting the minimum point (large alpha). You may have to experiemnt with few alphas to find something that works. When implementing you need to simulataneously compute the values of theta0 and theta1. (do not use a changed theta0 value in computing theta1 in the same iteration.)

In [ ]:
def gd2(obsX, obsY, alpha, threshold):
    """
    Input : observed vectors X, Y, alpha and threshold
    Return theta0, theta1 from Gradient Descent L2 loss algorithm
    Return: Iterations and L2 Error
    """
    iterations = 0
    theta0 = 0
    theta1 = 0
    n0 = 0
    n1 = 0
    oldError = -1
    newError = sqerror(obsX, obsY, n0, n1)
    while abs(newError - oldError) >= threshold:
        oldError = sqerror(obsX, obsY, n0, n1)
        theta0 = n0
        theta1 = n1
        n0 = theta0 - alpha*float(sum(theta1*obsX+theta0-obsY) / len(obsX))
        n1 = theta1 - alpha*float(sum((theta1*obsX+theta0-obsY)*obsX) / len(obsX))
        newError = sqerror(obsX, obsY, n0, n1)
        iterations += 1
        print(theta0, theta1) #print out the theta0 and theta1 values for each iteration in your function
    return n0, n1, newError, iterations
[theta0,theta1,newError,iterations] = gd2(x[0],y,0.01,0.0001)
print(iterations)

In [ ]:
theta0, theta1

In [ ]:
def gdh(obsX, obsY, alpha, threshold, delta):
    """
    Input : observed vectors X, Y, alpha and threshold
    Return theta0, theta1 from Gradient Descent huber loss algorithm
    Return: Iterations and huber Error
    """
    iterations = 0
    oldError = -1
    newError = 0
    theta0 = 0
    theta1 = 0
    while abs(newError - oldError) >= threshold:
        oldError = huberror(obsX, obsY, theta0, theta1, delta)
        val = h(theta0,theta1,obsX)-obsY
        theta0 = theta0 - alpha*float(sum(val/(val**2/delta**2+1)**0.5)/len(obsX))
        theta1 = theta1 - alpha*float(sum(obsX*val/(val**2/delta**2+1)**0.5)/len(obsX))
        newError = huberror(obsX, obsY, theta0, theta1, delta)
        iterations += 1
        if(iterations % 10 == 0):
            print(theta0, theta1) #print out the theta0 and theta1 values for each iteration in your function
    return theta0, theta1, newError, iterations
[theta0,theta1,newError,iterations] = gdh(x[0],y,0.01,0.000001,0.01)
print(iterations)

1. 
> theta0 = 0.23060509068378196

> theta1 = 0.13141113134375276

> alpha = 0.01

> error = 0.002097

2.

> The widget shows the same or similar listings, and the huber error is similar to the errors found through the gdh function.

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
result = lm.fit(x[0],y)
print(result.intercept_)
print(result.coef_)

In [ ]:
theta0 = result.intercept_
theta1 = result.coef_
sqerror(x[0],y,theta0,theta1)

# Extending the Model to a Bivariate
In this task we extend the model to predict housing price using two features "$x_1 = $Avg. Area House Age" and "$x_2 = $Avg. Area Number of Rooms". The regression model is then defined by  
$$
y = \theta_2*x_2 + \theta_1*x_1 + \theta_0
$$

In [ ]:
def gd22(obsX, obsY, alpha, threshold):
    """
    Input : observed vectors X, Y, alpha and threshold
    Return theta0, theta1, theta2 from Gradient Descent L2 loss algorithm
    Return: Iterations and L2 Error
    """
    iterations = 0
    y = lambda x1, x2, t0, t1, t2: t2*x2+t1*x1+t0
    theta0 = 0
    theta1 = 0
    theta2 = 0
    oldError = -1
    newError = 0
    while abs(newError - oldError) >= threshold:
        oldError = np.mean(np.square((theta2*obsX[1] + theta1*obsX[0] + theta0) - obsY))
        gd = (theta2*obsX[1] + theta1*obsX[0] + theta0) - obsY
        theta0 = theta0 - alpha*float(sum(gd) / len(obsX[0]))
        theta1 = theta1 - alpha*float(sum(gd*obsX[0]) / len(obsX[0]))
        theta2 = theta2 - alpha*float(sum(gd*obsX[1]) / len(obsX[0]))
        newError = np.mean(np.square(((theta2*obsX[1]+ theta1*obsX[0] + theta0) - obsY)))
        iterations += 1
    return theta0, theta1, theta2, newError, iterations
[theta0,theta1,theta2, newError,iterations] = gd22(x,y,0.01,0.0001)
print(iterations)
print(theta0, theta1,theta2)

Write the values of thetas obtained from function above.
$$\theta_0 = 0.29191222024889424$$ 
$$\theta_1 = 0.17880264543648708$$ 
$$\theta_2 = 0.1514344524598622$$ 
and write the model 
$$
y = 0.1514344524598622*x_2 + 0.17880264543648708*x_1 + 0.29191222024889424
$$

In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
y = df_training['Price']
X = df_training[["Avg. Area House Age", "Avg. Area Number of Rooms"]]
model = LinearRegression().fit(X, y)
model
print(model.coef_)
model.intercept_

In [ ]:
scaler = MinMaxScaler()
X0_scaled_values = scaler.fit_transform(df_testing.iloc[:,[1]])
X1_scaled_values = scaler.fit_transform(df_testing.iloc[:,[2]])
X_scaled_values = [X0_scaled_values,X1_scaled_values]
Y_scaled_values = scaler.fit_transform(df_testing.iloc[:,[5]])
res = 0.1514344524598622*X_scaled_values[1] + 0.17880264543648708*X_scaled_values[0] + 0.29191222024889424 #uses formula
error_gd = float(sum(abs(res - Y_scaled_values)) / len(Y_scaled_values)) #uses avg
print(error_gd)

In [ ]:
res = 0.99178836*X_scaled_values[1] + 0.95831457 *X_scaled_values[0] + 0.6755587973 #uses formula
error_lib = float(sum(abs(res - Y_scaled_values)) / len(Y_scaled_values)) # avg
print(error_lib)

They should be similar, since their error values are similar one should not have more disparity over the other.